In [ ]:
# pip install dotenv
# pip install anthropic
# pip install pyyaml

In [ ]:
import yaml
from pathlib import Path
from onyx import get_logger
from pathlib import Path
import re
from anthropic import Anthropic
from dotenv import load_dotenv
from onyx.rate_limiter import RateLimiter
from onyx.api_error_handler import ApiErrorHandler

logger = get_logger(__name__)

PROMPTS_DIR = Path("./prompts")

def get_prompt_and_system(prompt_name: str, **kwargs) -> tuple[str, str]:
    """Get both the system prompt and formatted user prompt as a tuple."""
    prompt_path = PROMPTS_DIR / f"{prompt_name}.yaml"
    if not prompt_path.exists():
        logger.error(f"Prompt file not found: {prompt_path}")
        raise FileNotFoundError(f"Prompt file not found: {prompt_path}")

    with open(prompt_path, "r", encoding="utf-8") as file:
        prompt_config = yaml.safe_load(file)

    system_prompt = prompt_config.get("system_prompt", "")
    template = prompt_config.get("prompt_template", "")

    try:
        formatted_prompt = template.format(**kwargs)
        return system_prompt, formatted_prompt
    except KeyError as e:
        logger.error(f"Missing required parameter in prompt {prompt_name}: {str(e)}")
        raise
    except Exception as e:
        logger.error(f"Error formatting prompt {prompt_name}: {str(e)}")
        raise

You are a member of an automated AI software engineering team. You will be asked to write code in various programming languages including Python, Swift, Javascript, and more.
- Use proper naming conventions for the programming language. 
- Always follow SOLID principles of programming.
- Write the minimum lines of code required to effectively accomplish your task, without sacrificing code readability.

Extensions by language:
  - Python = py
  - Javascript = js
  - Swift = swift

Your response will be parsed and used to create a file in the local directory, so it is crucial that you respond in the proper format.
Use level two headers (##) to indicate the file path (including the extension). Immediately follow it with a codeblock containing the content of the file.

<ResponseFormat>
 ## path/to/file.extension
```extension
[Your code here]
```
</ResponseFormat>



In [ ]:


def save_code_from_md(markdown: str, language: str, output_dir: Path) -> None:
    """
    Extract and save code blocks from markdown into project_dir / app_name / relative_path.language.

    Args:
        markdown: The original markdown content
        language: The language to extract from code blocks. Not case sensitive - lowercase enforced.
        output_dir: The parent directory that files and directories should be saved to.
    """
    sections = re.split(r"^##\s+", markdown, flags=re.MULTILINE)
    code_pattern = re.compile(
        rf"```{re.escape(language.lower())}\s*(.*?)```", re.DOTALL
    )
    count = 0
    for section in sections[1:]:
        lines = section.strip().split("\n", 1)
        if len(lines) < 2:
            continue
        rel_path, body = lines[0].lstrip("/"), lines[1]
        match = code_pattern.search(body)
        if not match:
            continue
        file_path = output_dir / rel_path
        file_path.parent.mkdir(parents=True, exist_ok=True)
        file_path.write_text(match.group(1).strip(), encoding="utf-8")
        count += 1
    logger.info(f"Parsed and saved {count} code blocks")


In [ ]:



def _send_prompt(
    prompt: str,
    system_prompt: str,
    temp: float = 0,
    thinking_enabled: bool = False
) -> str:
    """Send a prompt to Claude with retry logic for rate limits and other errors."""
    load_dotenv(override=True)
    response = ""

    
    if thinking_enabled:
        thinking_obj = {"type": "enabled", "budget_tokens": 24000}
        temp = 1
    else:
        thinking_obj = {"type": "disabled"}

    try:
        with Anthropic().messages.stream(
            model="claude-3-7-sonnet-20250219",
            max_tokens=128000,
            temperature=temp,
            thinking=thinking_obj,
            system=system_prompt,
            messages=[{"role": "user", "content": prompt}],
            extra_headers={"anthropic-beta": "output-128k-2025-02-19"},
        ) as stream:
            for chunk in stream:
                if hasattr(chunk, "type"):
                    if chunk.type == "text":
                        response += chunk.text

        return response

    except Exception as e:
        logger.error(f"Error in Claude API request: {str(e)}")
        raise

rate_limiter = RateLimiter(
    requests_per_minute=3, token_limit_per_minute=16000
)
error_handler = ApiErrorHandler(max_retries=5, base_wait=5, max_wait=120)

send_prompt = rate_limiter.with_rate_limiting(
    error_handler.with_retries(_send_prompt)
)



In [ ]:
lang = "python"
user_prompt = input("Enter prompt") ## Opens input field at top of screen

(system_prompt, prompt) = get_prompt_and_system(
    prompt_name="coder", language=lang, user_prompt=user_prompt
)

response = _send_prompt(
    prompt=prompt,
    system_prompt=system_prompt,
    thinking_enabled=False,
)

save_code_from_md(response, lang, output_dir=Path("./"))
print(response)

INFO     Parsed and saved 1 code blocks

## christmas_tree.py
```python
def draw_christmas_tree(height):
    """
    Draws a Christmas tree in the console with the specified height.
    
    Args:
        height (int): The height of the tree (number of rows for the tree part)
    """
    # Draw the tree part
    for i in range(height):
        # Calculate spaces before the stars
        spaces = height - i - 1
        # Calculate number of stars for this row
        stars = 2 * i + 1
        
        # Print the row
        print(' ' * spaces + '*' * stars)
    
    # Draw the trunk (2 rows)
    trunk_width = 3
    trunk_spaces = height - trunk_width // 2 - 1
    for _ in range(2):
        print(' ' * trunk_spaces + '*' * trunk_width)
    
    # Draw the base
    base_width = 5
    base_spaces = height - base_width // 2 - 1
    print(' ' * base_spaces + '*' * base_width)

if __name__ == "__main__":
    # Get tree height from user
    try:
        height = int(input("Enter the height of the Christmas tree: "))
        if heigh